This Notebook shows the code used to run LIME detections on images using YOLOv3 with a Darknet-53 backbone. Code used was from https://github.com/AntMorais/yolime.

Connect to google drive and setup path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path_setup = '/content/drive/MyDrive/'
img_folder = path_setup+"lime/LIME_images/"

In [ ]:
%cd /content/drive/MyDrive/lime

Clone repository to google drive

In [ ]:
!git clone https://github.com/AntMorais/yolime

Install and setup LIME

In [ ]:
!pip install lime

In [ ]:
%cd yolime
!make

/content/drive/MyDrive/lime/yolime
rm -rf ./obj/image_opencv.o ./obj/http_stream.o ./obj/gemm.o ./obj/utils.o ./obj/dark_cuda.o ./obj/convolutional_layer.o ./obj/list.o ./obj/image.o ./obj/activations.o ./obj/im2col.o ./obj/col2im.o ./obj/blas.o ./obj/crop_layer.o ./obj/dropout_layer.o ./obj/maxpool_layer.o ./obj/softmax_layer.o ./obj/data.o ./obj/matrix.o ./obj/network.o ./obj/connected_layer.o ./obj/cost_layer.o ./obj/parser.o ./obj/option_list.o ./obj/darknet.o ./obj/detection_layer.o ./obj/captcha.o ./obj/route_layer.o ./obj/writing.o ./obj/box.o ./obj/nightmare.o ./obj/normalization_layer.o ./obj/avgpool_layer.o ./obj/coco.o ./obj/dice.o ./obj/yolo.o ./obj/detector.o ./obj/layer.o ./obj/compare.o ./obj/classifier.o ./obj/local_layer.o ./obj/swag.o ./obj/shortcut_layer.o ./obj/activation_layer.o ./obj/rnn_layer.o ./obj/gru_layer.o ./obj/rnn.o ./obj/rnn_vid.o ./obj/crnn_layer.o ./obj/demo.o ./obj/tag.o ./obj/cifar.o ./obj/go.o ./obj/batchnorm_layer.o ./obj/art.o ./obj/region_layer.o

Load Images and Model Weights

In [ ]:
import argparse
import os
import glob
import random
import time
import cv2
import numpy as np
import darknet
import darknet_images


# No caso de querermos correr a rede default (para testar por ex.)
#txt_input = path_setup + "FIRELOC_DATA/train.txt"
#weights = path_setup + "FIRELOC_DATA/yolov4.weights"
#config_file = "cfg/yolov4.cfg"
#data_file = "cfg/coco.data"

# No caso de querermos a rede FireLoc
txt_input = "data/train.txt"
weights = path_setup + "/yolov3/darknet/backup/yolov3_custom_final.weights"

path_setup = "/content/drive/MyDrive/"
path_cfg = "/content/drive/MyDrive/yolov3/darknet/cfg/"
config_file = "cfg/yolov3_custom.cfg"
data_file = "data/obj.data"
# outras configurações
batch_size = True
dont_show = True
ext_output = True
save_labels = True
thresh = 0.3

# number of samples generated in explain_instance
num_samples = 20000
# max number of features to be used in the explanation (default is 100000)
num_features = 1000000


Choose which image to run LIME on

In [ ]:
images = darknet_images.load_images(txt_input)
image_name = images[0]
folder_name = image_name.replace("data/obj/","").replace(".jpg","")
#!mkdir img_foler/folder_name



Run all cells from here for each image

In [ ]:
network, class_names, class_colors = darknet.load_network(
        config_file,
        data_file,
        weights,
        batch_size
)


#---------------------------------------------------------------------------------------------

width = darknet.network_width(network)
height = darknet.network_height(network)
image = cv2.imread(image_name)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_resized = cv2.resize(image_rgb, (width, height),
                            interpolation=cv2.INTER_LINEAR)



def grey_out(image, x, y, w, h):
    # input: ndarray returned by imread, height and width
    #Create an array of object rect which represents the region of interest
    rect = [[x-w/2,y+h/2], [x-w/2,y-h/2], [x+w/2,y-h/2],[x+w/2,y+h/2]]
    mask = np.array([rect], dtype=np.int32)

    #Create a new array filled with zeros, size equal to size of the image to be filtered
    image2 = np.zeros((width, height), np.int8)

    cv2.fillPoly(image2, [mask],255)    
    maskimage2 = cv2.inRange(image2, 1, 255)
    out = cv2.bitwise_and(image, image, mask=maskimage2)
    return out



def lime_classification_function(image_numpy):

    probabilities = np.zeros((len(image_numpy), len(class_names)))
    for i in range(len(image_numpy)):
        img = image_numpy[i]
        image, detections = darknet_images.image_detection_lime(
                img, network, class_names,
                class_colors, thresh
                )

        if save_labels:
            darknet_images.save_annotations(image_name, image, detections, class_names)
        if not dont_show:
            cv2.imshow('Inference', image)
            if cv2.waitKey() & 0xFF == ord('q'):
                return
        with open("probabilityArray.txt","r") as prob_array:
            lines = [float(line.rstrip()) for line in prob_array]
            probabilities[i] = lines
    return probabilities


# input:
# - class_name: string with name of class we want to explain (e.g. "Dog", "Smoke" ...)
# output:
# - picked_class: index for the bounding box with most confidence of the specified input class 
# - detections
def get_most_confident_bbox(class_name):

    image_original, detections = darknet_images.image_detection(
                image_name, network, class_names,
                class_colors, thresh
                )
    darknet.print_detections(detections, ext_output)
    #detections is a list of tuples (class, confidence,(coordinates))
    picked_class = -1
    print(detections)
    for i in range(len(detections)):
        _tuple = detections[i]
        if _tuple[0] == class_name:
            coordinates = _tuple[2]
            with open("coordinates.txt","w") as coord_file:
                # we convert the coordinates to relative because that is how it is used in the C files
                coordinates_rel = darknet_images.convert2relative(image_original,coordinates)
                for _coord in coordinates_rel:
                    coord_file.write(str(_coord)+"\n")
                picked_class = i
            break
    if picked_class == -1:
        print("There is no " + class_name + " in this image!")
    return picked_class, detections


In [ ]:
class_name = 'polyp'
picked_class, coordinates = get_most_confident_bbox(class_name)

In [ ]:
# pass the image, the height and width
greyed_out_image = grey_out(image_resized, coordinates[picked_class][2][0], coordinates[picked_class][2][1],\
                            coordinates[picked_class][2][2], coordinates[picked_class][2][3])

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB))

In [ ]:
cv2_imshow(cv2.cvtColor(greyed_out_image, cv2.COLOR_BGR2RGB))
cv2.imwrite(img_folder+folder_name+"/cropped.png",cv2.cvtColor(greyed_out_image, cv2.COLOR_BGR2RGB))

In [ ]:
predictions_jpg = cv2.imread("predictions.jpg")
frame_normed = 255 * (frame - frame.min()) / (frame.max() - frame.min())
frame_normed = np.array(frame_normed, np.int)
cv2.imwrite(img_folder+ folder_name + "/predictions.jpg", predictions_jpg)
cv2_imshow(predictions_jpg)

In [ ]:
from lime import lime_image
#Objeto do tipo LimeImageExplainer
explainer = lime_image.LimeImageExplainer()
#Objeto do tipo ImageExplanation
explanation = explainer.explain_instance(np.array(image_resized), 
                                         lime_classification_function, # classification function
                                         top_labels=5, 
                                         hide_color=0, 
                                         num_samples=num_samples,
                                         num_features=num_features) # number of images that will be sent to classification function

  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
# mostra a imagem que acabámos de detetar
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
def show_bbox_centers(coordinates):

    import matplotlib.pyplot as plt
    #mostrar centros das bounding boxes 
    xx = []
    yy = []
    for class_coord in coordinates:
        xx.append(class_coord[2][0])
        yy.append(class_coord[2][1])
    plt.scatter(xx,\
            yy,\
            marker='o', color="red")
    
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0],\
    positive_only=False, num_features=5\
    ,hide_rest=False)
img_boundry2 = mark_boundaries(temp/255.0, mask)
plt.imshow(img_boundry2)
show_bbox_centers(coordinates)
plt.show()

In [ ]:
def crop_img(img, new_filename, coordinates, picked_class):
    [center_x, center_y, width, height] = [coord for coord in coordinates[picked_class][2]]
    top_left_x = round(center_x - width/2)
    top_left_y = round(center_y - height/2)
    print(top_left_x)
    print(top_left_y)
    print(height)
    print(width)
    if top_left_x < 0:
        top_left_x = 0
    if top_left_y < 0:
        top_left_y = 0
    crop_img = img[top_left_y:top_left_y+round(height), top_left_x:top_left_x+round(width)]
    plt.imshow(crop_img)
    plt.show()
    # guardo a nova imagem num ficheiro
    cv2.imwrite(new_filename,crop_img)



crop_img(img_boundry2, "explanationHere.jpg", coordinates, picked_class)